In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.1%2Bcu121-cp312-cp312-win_amd64.whl (2454.8 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.17.1%2Bcu121-cp312-cp312-win_amd64.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.2.1%2Bcu121-cp312-cp312-win_amd64.whl (4.0 MB)
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ------ --------------------------------- 0.4/2.6 MB 12.6 MB/s eta 0:00:01
     ----------- ---------------------------- 0.8/2.6 MB 9.6 MB/s eta 0:00:01
     ------------------- -------------------- 1.3/2.6 MB 10.2 MB/s eta 0:00:01
     ------------------------- -------------- 1.6/2.6 MB 10.5 MB/s eta 0:00:01
     ----------------------------------- ---- 2.3/2.6 MB 10.5 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 10.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated

In [ ]:
%pip install transformers datasets
%pip install torch==2.2.1+cu121
%pip install tensorflow

# from google.colab import drive
# drive.mount('/content/drive')

# %pip install kaggle
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'


In [3]:
import torch

DEVICE =  "cuda: 0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda: 0'

In [6]:
from datasets import load_dataset, Dataset, Value, ClassLabel, Features

my_dataset = load_dataset("./Emotion_Dataset", sep=",")
# Creating a ClassLabel Object
df = my_dataset["train"].to_pandas()
labels = ['sadness','joy','love','anger', 'fear', 'surprise']
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

my_dataset= my_dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
my_dataset = my_dataset.cast_column('label', ClassLabels)

my_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [30]:
X_train = my_dataset["train"].to_pandas()
X_val = my_dataset["validation"].to_pandas()
X_test = my_dataset["test"].to_pandas()
y_true = X_test.labels
y_true

0       0
1       0
2       0
3       1
4       0
       ..
1995    3
1996    3
1997    1
1998    1
1999    4
Name: labels, Length: 2000, dtype: int64

In [8]:
features = my_dataset["train"].features
features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [9]:
features["label"].int2str(0)

'sadness'

In [10]:
id2label = {idx:features["label"].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [11]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [12]:
X_train["label"].value_counts(normalize=True).sort_index()

label
0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: proportion, dtype: float64

In [13]:
from transformers import AutoTokenizer

model_ckpt = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [14]:
tokenizer(my_dataset["train"]["text"][:1])

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [15]:
def tokenize_text(examples):
  return tokenizer(examples["text"], truncation=True, max_length=512)


In [16]:
my_dataset = my_dataset.map(tokenize_text, batched=True)
my_dataset

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [18]:
class_weights = (1 - (X_train["label"].value_counts().sort_index() / len(X_train))).values #assigning high weight to the rare classes and low weight to the common classes to balance it out as there is an imbalance in the dataset
class_weights

import torch

class_weights = torch.from_numpy(class_weights).float()
class_weights = class_weights.to("cuda")
class_weights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], device='cuda:0')

In [19]:
my_dataset = my_dataset.rename_column("label", "labels")

In [20]:
from torch import nn
import torch
from transformers import Trainer

class WeightedLossTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    # Feed inputs to model and extract logits
    outputs = model(**inputs)
    logits = outputs.get("logits")
    # Extract labels
    labels = inputs.get("labels")
    #Define loss function with class weights
    loss_func=nn.CrossEntropyLoss(weight=class_weights)
    # Compute loss
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

In [21]:
from transformers import AutoModelForSequenceClassification, BertConfig, TextClassificationPipeline

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                          num_labels=6,
                                                          id2label=id2label,
                                                          label2id=label2id)
# config = BertConfig.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

# model = AutoModelForSequenceClassification.from_pretrained(config)

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

c:\Users\alice\Documents\Final year project\Fyp_code\.venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alice\.cache\huggingface\hub\models--microsoft--MiniLM-L12-H384-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from 

In [31]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  accuracy= accuracy_score(labels, preds)
  f1 = f1_score(labels, preds, average="weighted")
  return {"f1": f1, "accuracy" : accuracy}


In [32]:
%pip install transformers[torch]
%pip install accelerate -U
from transformers import TrainingArguments

batch_size = 64

logging_steps = len(my_dataset["train"])
output_dir = "/content/drive/MyDrive/Emotion_Output"
training_args = TrainingArguments(output_dir=output_dir,
                                  num_train_epochs=6,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  logging_steps=logging_steps,
                                  fp16=True)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
trainer = WeightedLossTrainer(model=model,
                              args=training_args,
                              compute_metrics=compute_metrics,
                              train_dataset=my_dataset["train"],
                              eval_dataset=my_dataset["validation"],
                              tokenizer=tokenizer)

c:\Users\alice\Documents\Final year project\Fyp_code\.venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [39]:
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [57]:
def predict(model, tokenizer):
    y_pred = []
    none_pred = []
    for i in tqdm(range(len(X_test))):
        # prompt = X_test.iloc[i]["text"].to(DEVICE)
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-classification",
                        model=model,
                        tokenizer=tokenizer,
                       )
        result = pipe(prompt)
        answer = result[0]['label'].split("=")[-1]
        #print(result)
        #print(answer)
        if "sadness" in answer:
            y_pred.append(0)
        elif "joy" in answer:
            y_pred.append(1)
        elif "love" in answer:
            y_pred.append(2)
        elif "anger" in answer:
            y_pred.append(3)
        elif "fear" in answer:
            y_pred.append(4)
        elif "surprise" in answer:
            y_pred.append(5)
        else:
            y_pred.append(6)
            none_pred.append(answer)
        # print(none_pred)
    return y_pred, none_pred

In [40]:
def evaluate(y_true, y_pred):

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels output: {0,1,2,3,4,5}

    # nested for loops
    for label in unique_labels:
        # will output a list of the index of one emotion at a time
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        # will output the list of one emotion
        label_y_true = [y_true[i] for i in label_indices]
        # label_y_true = [label for i in range(len(y_true))]
        # will output list of the predicted emotion in the same order as label_y_true
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [41]:
model.train() #changing the setting for training
trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 1.102960467338562, 'eval_f1': 0.5774674948660646, 'eval_accuracy': 0.6785, 'eval_runtime': 10.5353, 'eval_samples_per_second': 189.838, 'eval_steps_per_second': 3.037, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.736892819404602, 'eval_f1': 0.7934196114884992, 'eval_accuracy': 0.813, 'eval_runtime': 10.1848, 'eval_samples_per_second': 196.372, 'eval_steps_per_second': 3.142, 'epoch': 2.0}


...

  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.3562365770339966, 'eval_f1': 0.9142693775206736, 'eval_accuracy': 0.913, 'eval_runtime': 10.7352, 'eval_samples_per_second': 186.303, 'eval_steps_per_second': 2.981, 'epoch': 5.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.3322103023529053, 'eval_f1': 0.9195377183658051, 'eval_accuracy': 0.9185, 'eval_runtime': 10.8748, 'eval_samples_per_second': 183.912, 'eval_steps_per_second': 2.943, 'epoch': 6.0}
{'train_runtime': 1686.8367, 'train_samples_per_second': 56.911, 'train_steps_per_second': 0.889, 'train_loss': 0.6992425944010416, 'epoch': 6.0}


TrainOutput(global_step=1500, training_loss=0.6992425944010416, metrics={'train_runtime': 1686.8367, 'train_samples_per_second': 56.911, 'train_steps_per_second': 0.889, 'train_loss': 0.6992425944010416, 'epoch': 6.0})

In [42]:
model.eval() #changing the setting for testing
pred = trainer.predict(my_dataset["test"])
pred

  0%|          | 0/32 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 3.5019531 , -0.6020508 , -1.1777344 ,  0.1829834 , -0.9506836 ,
        -1.0439453 ],
       [ 3.5039062 , -0.6230469 , -1.1826172 ,  0.18188477, -0.94140625,
        -1.0253906 ],
       [ 3.4921875 , -0.69091797, -1.1669922 ,  0.15014648, -0.8881836 ,
        -0.9707031 ],
       ...,
       [-0.57910156,  3.6835938 ,  0.5600586 , -1.3583984 , -1.5244141 ,
        -0.86376953],
       [-0.6220703 ,  3.6679688 ,  0.5229492 , -1.3925781 , -1.4189453 ,
        -0.8286133 ],
       [-1.1816406 , -1.4697266 , -0.24572754, -0.70214844,  2.1386719 ,
         1.8916016 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 4], dtype=int64), metrics={'test_loss': 0.3069058954715729, 'test_f1': 0.9194852777281491, 'test_accuracy': 0.9175, 'test_runtime': 10.224, 'test_samples_per_second': 195.618, 'test_steps_per_second': 3.13})

In [43]:
trainer.evaluate(my_dataset["test"])

  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.3069058954715729,
 'eval_f1': 0.9194852777281491,
 'eval_accuracy': 0.9175,
 'eval_runtime': 10.4452,
 'eval_samples_per_second': 191.476,
 'eval_steps_per_second': 3.064,
 'epoch': 6.0}

In [58]:
y_pred, none_pred = predict(model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 2000/2000 [00:44<00:00, 45.16it/s]

Accuracy: 0.918
Accuracy for label 0: 0.966
Accuracy for label 1: 0.906
Accuracy for label 2: 0.950
Accuracy for label 3: 0.887
Accuracy for label 4: 0.879
Accuracy for label 5: 0.803

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       581
           1       0.97      0.91      0.94       695
           2       0.72      0.95      0.82       159
           3       0.94      0.89      0.91       275
           4       0.89      0.88      0.88       224
           5       0.68      0.80      0.74        66

    accuracy                           0.92      2000
   macro avg       0.86      0.90      0.88      2000
weighted avg       0.93      0.92      0.92      2000


Confusion Matrix:
[[561   5   3   8   4   0   0]
 [  1 630  54   4   0   6   0]
 [  0   8 151   0   0   0   0]
 [ 14   4   0 244  13   0   0]
 [  5   0   0   3 197  19   0]
 [  3   1   1   0   8  53   0]
 [  0   0   0   0   0   0   0]]
